## 对抗训练
![对抗样本](https://img-blog.csdnimg.cn/20210713144115288.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)



![对抗训练](https://img-blog.csdnimg.cn/20210716113626200.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

### 对抗训练方法
**Fast Gradient Method(FGM)**\
**Projected Gradient Descent(PGD)** 

In [1]:
!nvidia-smi

Mon May 30 06:32:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')
#设置路径
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


In [3]:
! pip install transformers==4.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 5.1 MB/s 
     |████████████████████████████████| 2.9 MB 39.5 MB/s 
     |████████████████████████████████| 880 kB 25.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=40a9d7bcf942cb2bb3b6105a011e86e2bc245818e74e7149e1b260fa19478e37
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [4]:
! pip install torch==1.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 753.4 MB 6.6 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.4.0 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.4.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.4.0 which is incompatible.


In [5]:
import torch
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

config = {
    'train_file_path':'/content/drive/MyDrive/Colab Notebooks/dataset/train.csv',
    'test_file_path':'/content/drive/MyDrive/Colab Notebooks/dataset/test.csv',
    'train_val_ratio':0.1,
    'model_path':'/content/drive/MyDrive/Colab Notebooks/dataset/NeZha_model',
    'batch_size':16,
    'head': 'CNN',
    'num_epochs':1,
    'warmup_ratio':0.1, # warm up
    'eps':0.1,    #对抗模型需要的参数
    'alpha':0.3,   #pgd需要的参数
    'adv':'fgm',   #对抗训练的方法
    'learning_rate':2e-5,
    'logging_step':500,
    'seed':2022
}

config['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'

def seed_everything(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  return seed

seed_everything(config['seed'])

2022

In [6]:
from collections import defaultdict
def read_data(config, tokenizer, mode = 'train'):
  data_df = pd.read_csv(config[f'{mode}_file_path'], sep=',')
  if mode == 'train':
    X_train, y_train = defaultdict(list),[]
    X_val, y_val = defaultdict(list),[]
    num_val = int(len(data_df) * config['train_val_ratio'])
  else:
    X_test, y_test = defaultdict(list),[]

  for i, row in tqdm(data_df.iterrows(), desc=f'preprocess {mode} data', colour = 'blue', total = len(data_df)):
    label = row[1] if mode == 'train' else 0
    sentence = row[-1]

    inputs = tokenizer.encode_plus(sentence, add_special_tokens = True, return_token_type_ids = True, return_attention_mask = True)

    if mode == 'train':
      if i < num_val:
        X_val['inputs_ids'].append(inputs['input_ids'])
        y_val.append(label)
        X_val['token_type_ids'].append(inputs['token_type_ids'])
        X_val['attention_mask'].append(inputs['attention_mask'])
      else:
        X_train['inputs_ids'].append(inputs['input_ids'])
        y_train.append(label)
        X_train['token_type_ids'].append(inputs['token_type_ids'])
        X_train['attention_mask'].append(inputs['attention_mask'])

    else:
        X_test['inputs_ids'].append(inputs['input_ids'])
        y_test.append(label)
        X_test['token_type_ids'].append(inputs['token_type_ids'])
        X_test['attention_mask'].append(inputs['attention_mask'])

  if mode == 'train':
    label2id = {label: i for i, label in enumerate(np.unique(y_train))}
    id2label = {i: label for label, i in label2id.items()}

    y_train = torch.tensor([label2id[i] for i in y_train], dtype =torch.long)

    y_val = torch.tensor([label2id[i] for i in y_val], dtype =torch.long)
    return X_train, y_train, X_val, y_val, label2id, id2label

  else:
    y_test = torch.tensor(y_test, dtype = torch.long)
    return X_test, y_test


In [7]:
from torch.utils.data import Dataset
class TNEWSData(Dataset):
  def __init__(self, X, y):
    self.x = X
    self.y = y
  
 
  def __getitem__(self, idx):
    return{
        'inputs_ids': self.x['inputs_ids'][idx],
        'label':self.y[idx],
        'token_type_ids':self.x['token_type_ids'][idx],
        'attention_mask':self.x['attention_mask'][idx]

    }

 
  def __len__(self):
    return self.y.size(0)

In [8]:
def collate_fn(example):
  input_ids_list = []
  labels = []
  token_type_ids_list = []
  attention_mask_list = []

  for ex in example:
    input_ids_list.append(ex['inputs_ids'])
    labels.append(ex['label'])
    token_type_ids_list.append(ex['token_type_ids'])
    attention_mask_list.append(ex['attention_mask'])

  max_len = max(len(input_ids) for input_ids in input_ids_list)
  input_ids_tensor = torch.zeros((len(labels), max_len),dtype=torch.long)
  token_type_ids_tensor = torch.zeros_like(input_ids_tensor)
  attention_mask_tensor = torch.zeros_like(input_ids_tensor)

  for i, input_ids in enumerate(input_ids_list):
    input_ids_tensor[i, :len(input_ids)] = torch.tensor(input_ids, dtype = torch.long)
    token_type_ids_tensor[i, :len(input_ids)] = torch.tensor(token_type_ids_list[i], dtype = torch.long)
    attention_mask_tensor[i, :len(input_ids)] = torch.tensor(attention_mask_list[i], dtype = torch.long)

  return {
      'input_ids': input_ids_tensor,
      'labels': torch.tensor(labels ,dtype= torch.long),
      'token_type_ids':token_type_ids_tensor,
      'attention_mask':attention_mask_tensor
  }  

In [9]:
from transformers import BertTokenizer
from torch.utils.data import DataLoader
def build_dataloader(config):
  tokenizer = BertTokenizer.from_pretrained(config['model_path'])
  X_train, y_train, X_val, y_val, label2id, id2label = read_data(config, tokenizer, mode='train')
  X_test, y_test = read_data(config, tokenizer, mode='test')

  train_dataset = TNEWSData(X_train, y_train)
  val_dataset = TNEWSData(X_val, y_val)
  test_dataset = TNEWSData(X_test, y_test)

  train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], num_workers = 4, shuffle = True, collate_fn=collate_fn)
  val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], num_workers = 4, shuffle = False, collate_fn=collate_fn)
  test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], num_workers = 4, shuffle = False, collate_fn=collate_fn)

  return train_dataloader, val_dataloader, test_dataloader, id2label


In [10]:
train_dataloader, val_dataloader, test_dataloader, id2label = build_dataloader(config)

preprocess test data: 100%|██████████| 10000/10000 [00:04<00:00, 2475.15it/s]


In [11]:
import torch.nn.functional as F
import torch.nn as nn
from NeZha import *
class NeZhaForTNEWS(NeZhaPreTrainedModel):
  def __init__(self, config, model_path, classifier):
    super(NeZhaForTNEWS, self).__init__(config)

    self.nezha = NeZhaModel.from_pretrained(model_path, config=config)
    self.classifier = classifier  # head
    self.config = config

  def forward(self, input_ids, token_type_ids, attention_mask, labels):
    outputs = self.nezha(input_ids = input_ids,
                token_type_ids = token_type_ids,
                attention_mask = attention_mask)
    hidden_states = outputs[2]

    logits = self.classifier(hidden_states, input_ids)

    outputs = (logits, )

    if labels is not None:
      loss_fct = FocalLoss(num_classes=self.config.num_labels)
      loss = loss_fct(logits, labels.view(-1))
      outputs = (loss, )+ outputs

    return outputs

In [12]:
from typing import List
class ConvClassifier(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.conv = nn.Conv1d(in_channels = config.hidden_size, out_channels = config.hidden_size, kernel_size = 3, padding=(3 - 1) // 2)
    self.global_max_pool = nn.AdaptiveMaxPool1d(1)
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.fc = nn.Linear(config.hidden_size, config.num_labels)

  def forward(self, hidden_states: List[torch.Tensor], input_ids: torch.Tensor):
    hidden_states = self.dropout(hidden_states[-1])
    hidden_states = hidden_states.permute(0 ,2, 1)

    out = F.relu(self.conv(hidden_states))
    out = self.global_max_pool(out).squeeze(dim=2)
    out = self.fc(out)
    
    return out

In [13]:
def build_model(model_path, config, head):
  heads = {
      'CNN': ConvClassifier
  }
  assert head in heads ,"head must have been implemented"
  print(f'>>> You are using {head} head, please wait...')
  model = NeZhaForTNEWS(config, model_path, heads[head](config))
  return model

In [14]:
from sklearn.metrics import f1_score
def evaluation(config, model, val_dataloader):
  model.eval()
  preds = []
  labels = []
  val_loss = 0.
  val_iterator = tqdm(val_dataloader, desc='Evaluation', total=len(val_dataloader))

  with torch.no_grad():
    for batch in val_iterator:
      labels.append(batch['labels'])
      batch = {item: value.to(config['device']) for item, value in batch.items()}
      loss, logits = model(**batch)[:2]
      val_loss += loss.item()
      preds.append(logits.argmax(dim = -1).detach().cpu())

  avg_val_loss = val_loss / len(val_dataloader)
  labels = torch.cat(labels, dim = 0).numpy()
  preds = torch.cat(preds, dim = 0).numpy()
  f1 = f1_score(labels, preds, average='macro')
  return avg_val_loss, f1

### 对抗样本：对人类看起来一样，对模型来说预测结果却完全不一样的样本。

## FGM-Fast Gradient Method
对于每个x:
1. 计算x的前向loss, 反向传播得到梯度；
2. 根据embeddign矩阵计算的梯度计算出r, 并加到当前embedding上，相当于x+r
3. 计算x+r的前向loss, 反向传播得到梯度，然后累加到(1)的梯度上；
4. 将embedding恢复为（1）时的embedding；
5. 根据（3）的梯度对参数进行更新。

## PGD-Projected Gradient Descent
FGM是一下子算出了对抗扰动，这样得到的扰动不一定是最优的。因此PGD进行了改进，多迭代了K(t)次，慢慢找到最优的扰动
对于每个x:

1.计算x的前向loss, 反向传播得到梯度；
  对于每步t：
  
  2. 根据embeddign矩阵计算的梯度计算出r, 并加到当前embedding上，相当于x+r；
  
  3. t如果不是最后一步，将梯度归0， 根据（2）的x+r计算前后向并得到梯度
  
  4. t是最后一步，恢复1的梯度，计算最后的x+r并将梯度累加到(1)上

5.将embedding恢复为（1）时的embedding；

6.根据（4）的梯度对参数进行更新。

In [15]:
from extra_loss import *
from extra_optim import *
from extra_fgm import *
from extra_pgd import *
from transformers import AdamW
from tqdm import trange


def train(config, id2label, train_dataloader, val_dataloader):
  nezha_config = NeZhaConfig.from_pretrained(config['model_path'])
  nezha_config.output_hidden_states = True
  nezha_config.num_labels = len(id2label)

  model = build_model(config['model_path'], nezha_config, config['head'])                                    

  

  # 得到模型的参数
  optimizer_grouped_parameters = model.parameters()
  # 定义一个基优化器
  optimizer = AdamW(model.parameters(), lr= config['learning_rate'])
  # Lookahead要有一个基优化器， k=5, alpha=1
  optimizer = Lookahead(optimizer, 5, 1)
  total_steps = config['num_epochs'] * len(train_dataloader)
  # 每调用warmup_steps次， 对应的学习率就会调整一次
  lr_scheduler = WarmupLinearSchedule(optimizer, warmup_steps = int(config['learning_rate'] * total_steps), t_total = total_steps)
  
  model.to(config['device'])                                                                 


### ----- adversarial -------#
  if config['adv'] == 'fgm':
    fgm = FGM(model)
  else:
    pgd = PGD(model)
    K = 3
### ----- adversarial -------#

  epoch_iterator = trange(config['num_epochs'])
  global_steps = 0
  train_loss = 0.
  logging_loss = 0.

  for epoch in epoch_iterator:
    train_iterator = tqdm(train_dataloader, desc='Training', total=len(train_dataloader))
    model.train()
    for batch in train_iterator:
      batch = {item: value.to(config['device']) for item, value in batch.items()}
      # 计算x的前项loss
      loss = model(**batch)[0]
      model.zero_grad()
      # 反向传播得到梯度
      loss.backward()

### ----- adversarial -------#
      if config['adv'] == 'fgm':
        # 计算x+r的前向loss, 反向传播得到梯度，然后累加到(1)的梯度上；
        fgm.attack(epsilon = config['eps'])
        # 计算x+r的前向loss
        loss_adv = model(**batch)[0]
        # 反向传播得到梯度，然后累加到(1)的梯度上；
        loss_adv.backward()
         #将embedding恢复为（1）时的embedding；
        fgm.restore()
      else:
        pgd.backup_grad()
        for t in range(K):
          fgm.attack(epsilon=config(['eps'], alpha=config['alpha'], is_first_attack=(t == 0)))
          if t != K - 1:
            model.zero_grad()
          else:
            pgd.restore_grad()
          loss_adv = model(**batch)[0]
          loss_adv.backward()
        pgd.restore()
### ----- adversarial -------#

      optimizer.step()
      lr_scheduler.step()

      train_loss += loss.item()
      global_steps += 1

      if global_steps % config['logging_step'] == 0:
        print_train_loss = (train_loss - logging_loss) / config['logging_step']
        logging_loss = train_loss
        avg_val_loss, f1 = evaluation(config, model, val_dataloader)

        print_log = f'>>>traing loss:{print_train_loss: .5f}, valid loss:{avg_val_loss: .5f}, valid f1 score:{f1: .5f}'
        print(print_log)
        model.train()

  return model    

In [16]:
model = train(config, id2label, train_dataloader, val_dataloader)

>>> You are using CNN head, please wait...


Some weights of NeZhaModel were not initialized from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/dataset/NeZha_model and are newly initialized: ['bert.encoder.layer.0.attention.self.relative_positions_encoding.positions_encoding', 'bert.encoder.layer.1.attention.self.relative_positions_encoding.positions_encoding', 'bert.encoder.layer.2.attention.self.relative_positions_encoding.positions_encoding', 'bert.encoder.layer.3.attention.self.relative_positions_encoding.positions_encoding', 'bert.encoder.layer.4.attention.self.relative_positions_encoding.positions_encoding', 'bert.encoder.layer.5.attention.self.relative_positions_encoding.positions_encoding', 'bert.encoder.layer.6.attention.self.relative_positions_encoding.positions_encoding', 'bert.encoder.layer.7.attention.self.relative_positions_encoding.positions_encoding', 'bert.encoder.layer.8.attention.self.relative_positions_encoding.positions_encoding', 'bert.encoder.layer.9.attention.self.relative_positions_encodi

>>>traing loss: 0.92146, valid loss: 0.71168, valid f1 score: 0.47703



Training:  33%|███▎      | 999/3002 [04:26<08:33,  3.90it/s]

Evaluation:   0%|          | 0/334 [00:00<?, ?it/s]

Evaluation:   0%|          | 1/334 [00:00<01:22,  4.05it/s]

Evaluation:   1%|          | 4/334 [00:00<00:27, 12.00it/s]

Evaluation:   2%|▏         | 7/334 [00:00<00:19, 17.14it/s]

Evaluation:   3%|▎         | 10/334 [00:00<00:16, 20.01it/s]

Evaluation:   4%|▍         | 13/334 [00:00<00:14, 21.58it/s]

Evaluation:   5%|▍         | 16/334 [00:00<00:13, 23.11it/s]

Evaluation:   6%|▌         | 19/334 [00:00<00:13, 23.98it/s]

Evaluation:   7%|▋         | 22/334 [00:01<00:12, 24.45it/s]

Evaluation:   7%|▋         | 25/334 [00:01<00:13, 23.66it/s]

Evaluation:   8%|▊         | 28/334 [00:01<00:12, 24.22it/s]

Evaluation:   9%|▉         | 31/334 [00:01<00:12, 23.67it/s]

Evaluation:  10%|█         | 34/334 [00:01<00:12, 23.28it/s]

Evaluation:  11%|█         | 37/334 [00:01<00:12, 23.47it/s]

Evaluation:  12%|█▏        | 40/334 [00:01<00:12, 24.23it/s]

Evaluation:  13%|█▎

>>>traing loss: 0.70314, valid loss: 0.65117, valid f1 score: 0.49247


...


Training:  83%|████████▎ | 2499/3002 [11:40<02:09,  3.89it/s]

Evaluation:   0%|          | 0/334 [00:00<?, ?it/s]

Evaluation:   0%|          | 1/334 [00:00<01:21,  4.08it/s]

Evaluation:   1%|          | 3/334 [00:00<00:33,  9.90it/s]

Evaluation:   2%|▏         | 6/334 [00:00<00:20, 15.80it/s]

Evaluation:   3%|▎         | 9/334 [00:00<00:16, 19.51it/s]

Evaluation:   4%|▎         | 12/334 [00:00<00:15, 21.36it/s]

Evaluation:   4%|▍         | 15/334 [00:00<00:14, 22.71it/s]

Evaluation:   5%|▌         | 18/334 [00:00<00:13, 24.02it/s]

Evaluation:   6%|▋         | 21/334 [00:01<00:12, 24.17it/s]

Evaluation:   7%|▋         | 24/334 [00:01<00:13, 23.81it/s]

Evaluation:   8%|▊         | 27/334 [00:01<00:12, 24.14it/s]

Evaluation:   9%|▉         | 30/334 [00:01<00:12, 23.95it/s]

Evaluation:  10%|▉         | 33/334 [00:01<00:12, 23.20it/s]

Evaluation:  11%|█         | 36/334 [00:01<00:12, 23.66it/s]

Evaluation:  12%|█▏        | 39/334 [00:01<00:12, 24.27it/s]

Evaluation:  13%|█▎

>>>traing loss: 0.59270, valid loss: 0.59022, valid f1 score: 0.53475



Training: 100%|█████████▉| 2999/3002 [14:03<00:00,  4.08it/s]

Evaluation:   0%|          | 0/334 [00:00<?, ?it/s]

Evaluation:   0%|          | 1/334 [00:00<01:21,  4.09it/s]

Evaluation:   1%|          | 3/334 [00:00<00:33,  9.92it/s]

Evaluation:   2%|▏         | 6/334 [00:00<00:20, 15.74it/s]

Evaluation:   3%|▎         | 9/334 [00:00<00:16, 19.59it/s]

Evaluation:   4%|▎         | 12/334 [00:00<00:14, 21.49it/s]

Evaluation:   4%|▍         | 15/334 [00:00<00:13, 22.94it/s]

Evaluation:   5%|▌         | 18/334 [00:00<00:13, 23.96it/s]

Evaluation:   6%|▋         | 21/334 [00:01<00:12, 24.37it/s]

Evaluation:   7%|▋         | 24/334 [00:01<00:12, 24.42it/s]

Evaluation:   8%|▊         | 27/334 [00:01<00:12, 24.51it/s]

Evaluation:   9%|▉         | 30/334 [00:01<00:12, 24.26it/s]

Evaluation:  10%|▉         | 33/334 [00:01<00:12, 23.36it/s]

Evaluation:  11%|█         | 36/334 [00:01<00:12, 23.87it/s]

Evaluation:  12%|█▏        | 39/334 [00:01<00:12, 24.30it/s]

Evaluation:  13%|█▎

>>>traing loss: 0.56590, valid loss: 0.57600, valid f1 score: 0.54563



100%|██████████| 1/1 [14:18<00:00, 858.66s/it]


In [17]:
def prediction(config, id2label, model, test_dataloader):
  test_iterator = tqdm(test_dataloader, desc='Prediction', total = len(test_dataloader))
  model.eval()
  test_preds = []

  with torch.no_grad():
    for batch in test_iterator:
      batch = {item: value.to(config['device']) for item, value in batch.items()}
      logits = model(**batch)[1]
      test_preds.append(logits.argmax(dim=-1).detach().cpu())
  
  test_preds = torch.cat(test_preds, dim=0).numpy()
  test_preds = [id2label[id_] for id_ in test_preds]

  test_df = pd.read_csv(config['test_file_path'], sep=',')
  test_df.insert(1, column='label', value=test_preds)
  test_df.drop(['sentence'], 1,inplace=True)#1表示按列删除
  test_df.to_csv('submission_Nezha_adversarial_training.csv', index=False, encoding= 'utf8')

In [18]:
prediction(config, id2label, model, test_dataloader)

Prediction: 100%|██████████| 625/625 [00:25<00:00, 24.84it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
